# Mini Project - API
###### Author: Ashley Jiangyang

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '5PcSbNzoasAV27shj1kJ'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url =  "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-11-21&end_date=2017-11-21&api_key={}".format(API_KEY)
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-11-21', 'end_date': '2017-11-21', 'frequency': 'daily', 'data': [['2017-11-21', 47.25, 48.59, 46.78, 48.39, None, 78502.0, 3782098.0, None, None, None]], 'collapse': None, 'order': None}}


Let's walk through the tasks for this mini project.

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
# colloect the whole year 2017 data
url_2017 =  "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={}".format(API_KEY)
r_2017 = requests.get(url_2017)

2. Convert the returned JSON object into a Python dictionary.

In [6]:
# Assigne to 'FSE_2017' and print the object type
FSE_2017 = r_2017.json()
print(type(FSE_2017))

<class 'dict'>


3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
# Assign the first day's open price as the highest and lowest value
high, low = FSE_2017['dataset_data']['data'][0][1], FSE_2017['dataset_data']['data'][0][1]

# Loop over the entire dataset, keep replacing the highest and lowest value if the loop find one
for item in FSE_2017['dataset_data']['data']:
    if item[1] == None:
        continue
    elif item[1] > high:
        high = item[1]
    if item[1] == None:
        continue
    elif item[1] < low:
        low = item[1]


#Print the final value
print("The highest opening price for the stock in this period: {}".format(high),
      "The lowest opening price for the stock in this period: {}".format(low), 
      sep = '\n')

The highest opening price for the stock in this period: 53.11
The lowest opening price for the stock in this period: 34.0


4. What was the largest change in any one day (based on High and Low price)?

In [8]:
# Intinal the greatest change as the high - low price as of the first day
max_change = FSE_2017['dataset_data']['data'][0][2] - FSE_2017['dataset_data']['data'][0][3]

# Replacing the value of greatest change if the for loop finds one
for item in FSE_2017['dataset_data']['data']:
    if item[2] == None or item[3] == None:
        continue
    elif item[2] - item[3] > max_change:
        max_change = item[2] - item[3]

#Print the final value
print('the largest change in any one day: {:.3}'.format(max_change))

the largest change in any one day: 2.81


5. What was the largest change between any two days (based on Closing Price)?

In [9]:
# use zip to map the value of every other day and get differences between every two other days
closing_diff = []
for x, y in zip(FSE_2017['dataset_data']["data"][0::], FSE_2017['dataset_data']["data"][2::]):
    if y[4] == None or x[4] == None:
        continue
    closing_diff.append(abs(y[4]-x[4]))
max_closing_diff = max(closing_diff)

#print the max value
print('The largest change between any two days: {:.3}'.format(max_closing_diff))

The largest change between any two days: 3.15


6. What was the average daily trading volume during this year?

In [10]:
# use for loop to count all the records and accumulate the Volume
count, total = 0, 0
for item in FSE_2017['dataset_data']['data']:    
    if item[6] == None:
        continue
    count += 1
    total += item[6]
    
average = total/count
total
count
#print the average value
print('The average daily trading volume dring this year: {}'.format(round(average,2)))

The average daily trading volume dring this year: 89124.34


7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [11]:
#Use list comprehension and 'sort()' to create the list for volume of the year
volume=[item[6] for item in FSE_2017['dataset_data']['data'] if item[6] != None]
volume.sort()

# Define a function to get the median
def findMedian(volume):    
    median = float(len(volume))/2
    if median % 2 != 0:
        return volume[int(median - 0.5)]
    else:
        return (volume[int(median)] + volume[int(median - 1)])/2   
    
# Print the calculated median
print('The median trading volume during this year: {}'.format(findMedian(volume)))

The median trading volume during this year: 76286.0
